# Purpose

Create a side-by-side image viewer with ipywidget button to make plotted region the same.

# Imports

In [1]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget
from IPython.display import display, display_html, HTML

import cv2
import imutils
from PIL import Image

import pandas as pd
from scipy import stats

import sys
sys.executable

'/Users/nordin/python_envs/voila_opencv/.venv/bin/python'

# Code

In [2]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='auto',
        padding='5px 5px 5px 5px'
     )


class SideBySideImageViewer(widgets.VBox):
    
    def __init__(self, img_l, img_r, title_left='', title_right='', vmin=0, vmax=None):
        
        super().__init__()
        output = widgets.Output()
        self.image_left = img_l
        self.image_right = img_r
        self.title_left = title_left
        self.title_right = title_right
        if vmax:
            self.vmax_initial = vmax
        else:
            self.vmax_initial = np.max(self.image_left)

        with output:
            self.fig, self.ax = plt.subplots(ncols=2, constrained_layout=True, figsize=(10, 3.5))
        self.ax_left, self.ax_right = self.ax
        
        self.axesimage_left = self.ax_left.imshow(
            self.image_left, interpolation=None, cmap="gray", vmin=vmin, vmax=self.vmax_initial
        )
        self.ax_left.set_title(self.title_left)
        self.axesimage_right = self.ax_right.imshow(
            self.image_right, interpolation=None, cmap="gray", vmin=vmin, vmax=self.vmax_initial
        )
        self.ax_right.set_title(self.title_right)
       
        style_HTML = {'description_width': 'initial'}
        self.min_max_text_left = widgets.HTML(
            value=f"{np.min(self.image_left)}, {np.max(self.image_left)}",
            placeholder="Some HTML",
            description="Left image min, max:",
            style=style_HTML,
        )
        
        self.min_max_text_right = widgets.HTML(
            value=f"{np.min(self.image_right)}, {np.max(self.image_right)}",
            placeholder="Some HTML",
            description="Right image min, max:",
            style=style_HTML,
        )

        self.vmax_slider = widgets.IntSlider(
            value=self.vmax_initial,
            min=0,
            max=1023,
            step=1,
            description="vmax",
            continuous_update=False,
        )

        self.vmin_slider = widgets.IntSlider(
            value=vmin,
            min=0,
            max=1023,
            step=1,
            description="vmin",
            continuous_update=False,
        )
        
        self.xylimits_button_right_to_left = widgets.Button(
            description='Copy image x,y limits to left image',
            tooltip='Make left image x,y limits the same as the right image',
            # button_style='info',
            layout=widgets.Layout(width='300px')
        )
        self.xylimits_button_right_to_left.on_click(self.xylimits_right_to_left_button_click)
        self.xylimits_button_left_to_right = widgets.Button(
            description='Copy image x,y limits to right image',
            tooltip='Make right image x,y limits the same as the left image',
            # button_style='info',
            layout=widgets.Layout(width='300px')
        )
        self.xylimits_button_left_to_right.on_click(self.xylimits_left_to_right_button_click)
        
        self.right_xylim = widgets.HTML(
            value=f"{self.ax_right.get_xlim()}, {self.ax_right.get_ylim()}",
            placeholder="Some HTML",
            description="xlim, ylim:",
            style=style_HTML,
            layout=widgets.Layout(margin='auto'),
        )
        self.left_xylim = widgets.HTML(
            value=f"{self.ax_left.get_xlim()}, {self.ax_left.get_ylim()}",
            placeholder="Some HTML",
            description="xlim, ylim:",
            style=style_HTML,
            layout=widgets.Layout(margin='auto'),
        )

        controls = widgets.HBox(
            [
                widgets.VBox(
                    [
                        self.vmax_slider,
                        self.vmin_slider,
                        widgets.HBox(
                            [
                                self.min_max_text_left,
                                widgets.HTML(value='&nbsp;'*80),
                                self.min_max_text_right,
                            ]
                        )
                    ],
                ),
            ],
            layout=widgets.Layout(margin='auto', padding='5px 5px 5px 5px')
        )
        
        range_stuff = widgets.HBox(
            [
                widgets.VBox([self.xylimits_button_left_to_right, self.left_xylim], layout=make_box_layout()),
                widgets.HTML(value='&nbsp;'*30),
                widgets.VBox([self.xylimits_button_right_to_left, self.right_xylim], layout=make_box_layout())
            ],
            layout=widgets.Layout(margin='auto', padding='5px 5px 5px 5px')
        )

        # observe stuff
        self.vmax_slider.observe(self.update_vmax, "value")
        self.vmin_slider.observe(self.update_vmin, "value")

        # add to children
        self.children = [controls, output, range_stuff]
        
    def update_vmax(self, change):
        if change.new <= self.vmin_slider.value:
            # Don't allow a change in slider value if <vmin
            self.vmax_slider.value = change.old
        else:
            self.axesimage_left.set_clim(vmax=change.new)
            self.axesimage_right.set_clim(vmax=change.new)
            self.fig.canvas.draw()
         
    def update_vmin(self, change):
        if change.new >= self.vmax_slider.value:
            # Don't allow a change in slider value if >vmax
            self.vmin_slider.value = change.old
        else:
            self.axesimage_left.set_clim(vmin=change.new)
            self.axesimage_right.set_clim(vmin=change.new)
            self.fig.canvas.draw()
        
    def xylimits_right_to_left_button_click(self, value):
        xlim = self.ax_right.get_xlim()
        ylim = self.ax_right.get_ylim()
        self.ax_left.set_xlim(*xlim)
        self.ax_left.set_ylim(*ylim)
        self.fig.canvas.draw()
        self.update_xylim_display()
        
    def xylimits_left_to_right_button_click(self, value):
        xlim = self.ax_left.get_xlim()
        ylim = self.ax_left.get_ylim()
        self.ax_right.set_xlim(*xlim)
        self.ax_right.set_ylim(*ylim)
        self.fig.canvas.draw()
        self.update_xylim_display()
        
    def update_xylim_display(self):
        self.left_xylim.value = "({:.1f}, {:.1f}), ({:.1f}, {:.1f})".format(*self.ax_left.get_xlim(), *self.ax_left.get_ylim())
        self.right_xylim.value = "({:.1f}, {:.1f}), ({:.1f}, {:.1f})".format(*self.ax_right.get_xlim(), *self.ax_right.get_ylim())

# Example

## Create images

In [3]:
x = np.linspace(-3.2, 3.2, 1600)
y = np.linspace(-2.4, 2.4, 1200)
num_y, num_x = len(y), len(x)

# Centered square
img1 = np.zeros((len(y), len(x)), dtype='uint16')
size = 100
img1[num_y//2 - size//2: num_y//2 + size//2, num_x//2 - size//2: num_x//2 + size//2] = 1000

# Centered rectangle
img2 = np.zeros((len(y), len(x)), dtype='uint16')
size_y, size_x = 200, 80
img2[num_y//2 - size_y//2: num_y//2 + size_y//2, num_x//2 - size_x//2: num_x//2 + size_x//2] = 500

## Side-by-side images

Procedure:

- Click on rectangle selection tool in toolbar
- Go to right image and select a region
- Click button "Left image range = Right image range"

Outcome: left image range will be the same as the right image range

In [4]:
SideBySideImageViewer(img1, img2, title_left='img1', title_right='img2')

SideBySideImageViewer(children=(HBox(children=(VBox(children=(IntSlider(value=1000, continuous_update=False, d…